<h1><center> PPOL564 - Data Science I: Foundations <br><br><font color='grey'> More on Numpy</font></center><h1>

**In this Notebook we cover**

- Stacking 
- Broadcasting
- Vectorization 
- Handling Multiple Data Types in `numpy` arrays

In [2]:
import numpy as np 
import math

## Stacking

We can easily stack and grow numpy arrays

In [4]:
m1 = np.random.randn(10).reshape(5,-1).round(1)
m2 = np.random.poisson(1,10).reshape(5,-1)

In [5]:
m1

array([[-0.6,  0.1],
       [-1.1,  0.2],
       [ 0.1,  0.4],
       [-0.6, -1.5],
       [-0.2, -0.1]])

In [6]:
m2

array([[2, 1],
       [0, 1],
       [2, 0],
       [0, 1],
       [1, 2]])

#### `rbind`: binding row

In [9]:
# stack the two columns using concatenate
np.concatenate([m1,m2],axis=0)

array([[-0.6,  0.1],
       [-1.1,  0.2],
       [ 0.1,  0.4],
       [-0.6, -1.5],
       [-0.2, -0.1],
       [ 2. ,  1. ],
       [ 0. ,  1. ],
       [ 2. ,  0. ],
       [ 0. ,  1. ],
       [ 1. ,  2. ]])

In [10]:
# or use verticle stack
np.vstack([m1,m2])

array([[-0.6,  0.1],
       [-1.1,  0.2],
       [ 0.1,  0.4],
       [-0.6, -1.5],
       [-0.2, -0.1],
       [ 2. ,  1. ],
       [ 0. ,  1. ],
       [ 2. ,  0. ],
       [ 0. ,  1. ],
       [ 1. ,  2. ]])

#### `cbind`: binding columns

In [11]:
np.concatenate([m1,m2],axis=1)

array([[-0.6,  0.1,  2. ,  1. ],
       [-1.1,  0.2,  0. ,  1. ],
       [ 0.1,  0.4,  2. ,  0. ],
       [-0.6, -1.5,  0. ,  1. ],
       [-0.2, -0.1,  1. ,  2. ]])

In [15]:
M = np.hstack([m1,m2])
M

array([[-0.6,  0.1,  2. ,  1. ],
       [-1.1,  0.2,  0. ,  1. ],
       [ 0.1,  0.4,  2. ,  0. ],
       [-0.6, -1.5,  0. ,  1. ],
       [-0.2, -0.1,  1. ,  2. ]])

## Views vs. Copies

Note that when we slice an array we **_do not copy the array_**, rather we get a "**view**" of the array. 

In [16]:
# Recall the behavior of double assignment with lists
x = [1,2,3]
y = x
y[2] = 100
x

[1, 2, 100]

In [17]:
# We can get around this behavior by making copies. 
# One way to make a copy is to slice
y = x[:]
y[2] = -999
x

[1, 2, 100]

When we slice an array, we get a sub-"view" of the data that still effects the original data object.

In [18]:
P = np.ones((5,5))
P

array([[1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1.]])

In [19]:
g = P[:2,:2] 
g

array([[1., 1.],
       [1., 1.]])

In [20]:
g += 100
g

array([[101., 101.],
       [101., 101.]])

In [21]:
P

array([[101., 101.,   1.,   1.,   1.],
       [101., 101.,   1.,   1.,   1.],
       [  1.,   1.,   1.,   1.,   1.],
       [  1.,   1.,   1.,   1.,   1.],
       [  1.,   1.,   1.,   1.,   1.]])

As [noted in the reading](https://jakevdp.github.io/PythonDataScienceHandbook/02.02-the-basics-of-numpy-arrays.html): 

> "This **_default behavior is actually quite useful_**: it means that when we work with large datasets, we can access and process pieces of these datasets without the need to copy the underlying data buffer."

To get around this behavior, we again just need to make a `.copy()`. 

In [22]:
g2 = P[:2,:2].copy()
g2 -= 1000
g2

array([[-899., -899.],
       [-899., -899.]])

In [23]:
P

array([[101., 101.,   1.,   1.,   1.],
       [101., 101.,   1.,   1.,   1.],
       [  1.,   1.,   1.,   1.,   1.],
       [  1.,   1.,   1.,   1.,   1.],
       [  1.,   1.,   1.,   1.,   1.]])

# Broadcasting 

**Broadcasting** makes it possible for operations to be performed on arrays of mismatched shapes.

Broadcasting describes how numpy treats arrays with different shapes during arithmetic operations. Subject to certain constraints, the smaller array is "broadcast" across the larger array so that they have compatible shapes.

For example, say we have a numpy array of dimensions (5,1)

$$ 
\begin{bmatrix} 1\\2\\3\\4\\5\end{bmatrix}
$$

Now say we wanted to add the values in this array by 5

$$ 
\begin{bmatrix} 1\\2\\3\\4\\5\end{bmatrix} + 5
$$

Broadcasting "pads" the array of 5 (which is shape = 1,1), and extends it so that it has similar dimension to the larger array in which the computation is being performed.

$$ 
\begin{bmatrix} 1\\2\\3\\4\\5\end{bmatrix} + \begin{bmatrix} 5\\\color{lightgrey}{5}\\\color{lightgrey}{5}\\\color{lightgrey}{5}\\\color{lightgrey}{5}\end{bmatrix}
$$

$$ 
\begin{bmatrix} 1 + 5\\2 + 5\\3 + 5\\4 + 5\\5 + 5\end{bmatrix} 
$$

$$ 
\begin{bmatrix} 6\\7\\8\\9\\10\end{bmatrix} 
$$

In [24]:
A = np.array([1,2,3,4,5])
A + 5

array([ 6,  7,  8,  9, 10])

By 'broadcast', we mean that the smaller array is made to match the size of the larger array in order to allow for element-wise manipulations.

### How it works:

- Shapes of the two arrays are compared _element-wise_. 
- Dimensions are considered in reverse order, starting with the trailing dimensions, and working forward 
- We are stretching the smaller array by making copies of its elements. `numpy` does not actually duplicate the smaller array; instead, it makes computationally efficient use of existing structures in memory that achieve the same result.

A general **Rule of thumb**: All corresponding dimension of the arrays must be compatible or one of the two dimensions is 1.

## Rules of Broadcasting

Broadcasting in NumPy follows a strict set of rules to determine the interaction between the two arrays (from [reading](https://jakevdp.github.io/PythonDataScienceHandbook/02.05-computation-on-arrays-broadcasting.html)):


### Rule 1
> If the two arrays differ in their number of dimensions, the shape of the one with fewer dimensions is padded with ones on its leading (left) side.

### Rule 2

> If the shape of the two arrays does not match in any dimension, the array with shape equal to 1 in that dimension is stretched to match the other shape.

### Rule 3 

> If in any dimension the sizes disagree and neither is equal to 1, an error is raised.

#### Example 1

In [25]:
np.arange(3) + 5

array([5, 6, 7])

$$
\texttt{np.arange(3)} = \begin{bmatrix} 0&1&2\end{bmatrix}
$$

<br> 

$$
\texttt{5}  = \begin{bmatrix} 5 \end{bmatrix}
$$

<br> 

$$
\begin{bmatrix} 0&1&2\end{bmatrix} + \begin{bmatrix} 5 & \color{lightgrey}{5} & \color{lightgrey}{5}\end{bmatrix} = \begin{bmatrix} 5 & 6 & 7\end{bmatrix} 
$$

#### Example 2

In [26]:
np.ones((3,3)) + np.arange(3)

array([[1., 2., 3.],
       [1., 2., 3.],
       [1., 2., 3.]])

$$
\texttt{np.ones((3,3)) = }\begin{bmatrix} 1 & 1 & 1\\ 1 & 1 & 1 \\ 1 & 1 & 1 \end{bmatrix}
$$

<br>

$$
\texttt{np.arange(3)} = \begin{bmatrix} 0 & 1 & 2\end{bmatrix} 
$$

<br>

$$
\begin{bmatrix} 1 & 1 & 1\\ 1 & 1 & 1 \\ 1 & 1 & 1 \end{bmatrix} + 
\begin{bmatrix} 0 & 1 & 2\\ \color{lightgrey}{0} & \color{lightgrey}{1} & \color{lightgrey}{2} \\  \color{lightgrey}{0} & \color{lightgrey}{1} & \color{lightgrey}{2}\end{bmatrix}  = 
\begin{bmatrix} 1 & 2 & 3\\ 1 & 2 & 3 \\ 1 & 2 & 3 \end{bmatrix} 
$$

#### Example 3

In [27]:
np.arange(3).reshape(3,1) + np.arange(3)

array([[0, 1, 2],
       [1, 2, 3],
       [2, 3, 4]])

$$
\texttt{np.arange(3).reshape(3,1)} = \begin{bmatrix} 0 \\ 1 \\ 2\end{bmatrix} 
$$

<br>

$$
\texttt{np.arange(3)} = \begin{bmatrix} 0 & 1 & 2\end{bmatrix} 
$$

<br>

$$
\begin{bmatrix} 0 & \color{lightgrey}{0} & \color{lightgrey}{0} \\ 1 & \color{lightgrey}{1} & \color{lightgrey}{1} \\  2 & \color{lightgrey}{2} & \color{lightgrey}{2}\end{bmatrix} +
\begin{bmatrix} 0 & 1 & 2\\ \color{lightgrey}{0} & \color{lightgrey}{1} & \color{lightgrey}{2} \\  \color{lightgrey}{0} & \color{lightgrey}{1} & \color{lightgrey}{2}\end{bmatrix}  =
\begin{bmatrix} 0 & 1 & 2\\ 1 &2&3 \\ 2& 3 & 4\end{bmatrix} 
$$


#### Example 4

Example of dimensional disagreement.

In [28]:
np.ones((4,7)) 

array([[1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1.]])

In [29]:
np.ones((4,7))  + np.zeros( (5,9) )

ValueError: operands could not be broadcast together with shapes (4,7) (5,9) 

In [30]:
np.ones((4,7))  + np.zeros( (1,7) )

array([[1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1.]])

# Vectorization

Similar to broadcasting, vectorization allows for simultaneous computation along all values in the array. 

In [31]:
X = np.random.randint(1,10,50).reshape(10,5)
X

array([[9, 2, 2, 4, 9],
       [4, 8, 8, 8, 4],
       [3, 8, 8, 2, 5],
       [4, 3, 9, 6, 1],
       [9, 2, 9, 8, 5],
       [2, 4, 8, 6, 3],
       [1, 6, 7, 2, 3],
       [8, 6, 1, 8, 3],
       [4, 6, 2, 6, 7],
       [1, 2, 6, 6, 4]])

In [32]:
np.log(X)

array([[2.19722458, 0.69314718, 0.69314718, 1.38629436, 2.19722458],
       [1.38629436, 2.07944154, 2.07944154, 2.07944154, 1.38629436],
       [1.09861229, 2.07944154, 2.07944154, 0.69314718, 1.60943791],
       [1.38629436, 1.09861229, 2.19722458, 1.79175947, 0.        ],
       [2.19722458, 0.69314718, 2.19722458, 2.07944154, 1.60943791],
       [0.69314718, 1.38629436, 2.07944154, 1.79175947, 1.09861229],
       [0.        , 1.79175947, 1.94591015, 0.69314718, 1.09861229],
       [2.07944154, 1.79175947, 0.        , 2.07944154, 1.09861229],
       [1.38629436, 1.79175947, 0.69314718, 1.79175947, 1.94591015],
       [0.        , 0.69314718, 1.79175947, 1.79175947, 1.38629436]])

The computations are performed on each element in the array _simultaneously_. 

> "When looping over an array or any data structure in Python, there’s a lot of overhead involved. Vectorized operations in `numpy` delegate the looping internally to highly optimized `C` and `Fortran` functions, making for cleaner and faster Python code." - [RealPython](https://realpython.com/numpy-array-programming/)

Again, let's consider what this same operation would need to look like if we were dealing with a nested list. We'd need to perform each operation element-by-element in the nested list structure. 

In [33]:
X2 = X.tolist()
n_rows = len(X2)
n_cols = len(X2[0])
for i in range(n_rows):
    for j in range(n_cols):
        X2[i][j] = math.log(X2[i][j])
X2

[[2.1972245773362196,
  0.6931471805599453,
  0.6931471805599453,
  1.3862943611198906,
  2.1972245773362196],
 [1.3862943611198906,
  2.0794415416798357,
  2.0794415416798357,
  2.0794415416798357,
  1.3862943611198906],
 [1.0986122886681098,
  2.0794415416798357,
  2.0794415416798357,
  0.6931471805599453,
  1.6094379124341003],
 [1.3862943611198906,
  1.0986122886681098,
  2.1972245773362196,
  1.791759469228055,
  0.0],
 [2.1972245773362196,
  0.6931471805599453,
  2.1972245773362196,
  2.0794415416798357,
  1.6094379124341003],
 [0.6931471805599453,
  1.3862943611198906,
  2.0794415416798357,
  1.791759469228055,
  1.0986122886681098],
 [0.0,
  1.791759469228055,
  1.9459101490553132,
  0.6931471805599453,
  1.0986122886681098],
 [2.0794415416798357,
  1.791759469228055,
  0.0,
  2.0794415416798357,
  1.0986122886681098],
 [1.3862943611198906,
  1.791759469228055,
  0.6931471805599453,
  1.791759469228055,
  1.9459101490553132],
 [0.0,
  0.6931471805599453,
  1.791759469228055,
  

Vectorization frees us from this tedium. Moreover, it's extremely efficient so we can perform computations quickly.

For example:

In [34]:
# Locate the absolute value for an array
np.abs([1,2,-6,7,8])

array([1, 2, 6, 7, 8])

In [35]:
# Round Values to the k-th decimal point
np.round(np.log(X),1)

array([[2.2, 0.7, 0.7, 1.4, 2.2],
       [1.4, 2.1, 2.1, 2.1, 1.4],
       [1.1, 2.1, 2.1, 0.7, 1.6],
       [1.4, 1.1, 2.2, 1.8, 0. ],
       [2.2, 0.7, 2.2, 2.1, 1.6],
       [0.7, 1.4, 2.1, 1.8, 1.1],
       [0. , 1.8, 1.9, 0.7, 1.1],
       [2.1, 1.8, 0. , 2.1, 1.1],
       [1.4, 1.8, 0.7, 1.8, 1.9],
       [0. , 0.7, 1.8, 1.8, 1.4]])

In [36]:
# Count the number of non zeros
np.count_nonzero(np.array([1,0,8,0,1]))

3

Numpy comes baked in with a large number of `ufuncs` (or "universal functions") that are all vectorized. [See here for a detailed list of these operations.](https://docs.scipy.org/doc/numpy-1.13.0/reference/routines.math.html)

## Vectorization across array dimensions

The universal functions constructed in Python come with an `axis` argument that outlines how the function should be applied

In [37]:
A = np.random.randint(1,10,100).reshape(20,5)
A

array([[9, 7, 2, 7, 6],
       [4, 4, 1, 9, 2],
       [4, 3, 4, 9, 5],
       [3, 1, 3, 2, 3],
       [5, 8, 7, 8, 2],
       [1, 5, 8, 2, 1],
       [8, 6, 1, 1, 2],
       [9, 6, 8, 7, 9],
       [6, 9, 2, 8, 3],
       [6, 8, 8, 7, 6],
       [5, 3, 2, 1, 7],
       [8, 2, 4, 4, 5],
       [8, 6, 3, 6, 8],
       [8, 7, 8, 7, 7],
       [7, 8, 2, 4, 4],
       [8, 8, 7, 5, 1],
       [5, 9, 3, 5, 8],
       [2, 6, 5, 4, 4],
       [5, 4, 2, 3, 1],
       [6, 7, 2, 7, 7]])

Consider calculating the average across some data set. By default, the ufunc `.mean()` will calculate the average for the entire data matrix.

In [38]:
A.mean()

5.13

If we wanted to calculate the mean for each observation (row) or variable (column), we'll need to use the `axis` argument to specify which. 

- `axis = 0` == move across the columns
- `axis = 1` == move across the rows

In [39]:
A.mean(axis=0)

array([5.85, 5.85, 4.1 , 5.3 , 4.55])

In [40]:
A.mean(axis=1)

array([6.2, 4. , 5. , 2.4, 6. , 3.4, 3.6, 7.8, 5.6, 7. , 3.6, 4.6, 6.2,
       7.4, 5. , 5.8, 6. , 4.2, 3. , 5.8])

## Building vectorized functions

Consider the following function that yields a different string when input `a` is larger/smaller than input `b`.

In [41]:
def bigsmall(a,b):
    if a > b:
        return "A is larger"
    else:
        return "B is larger"

In [42]:
bigsmall(5,6)

'B is larger'

In [43]:
bigsmall(6,5)

'A is larger'

We can implement this function in a vectorized fashion using the `np.vectorize()` method. 

In [44]:
# Create a vectorized version of the function
vec_bigsmall = np.vectorize(bigsmall)
vec_bigsmall 

In [45]:
# And now implement on arrays of numbers!
vec_bigsmall([0,2,5,7,0],[4,3,10,2,6])

array(['B is larger', 'B is larger', 'B is larger', 'A is larger',
       'B is larger'], dtype='<U11')

# Road to the DataFrame &rarr; Handling Multiple Data Types

Out of the box, numpy arrays can only handle one data class at a time...

In [46]:
x = np.array([1,2,3,4])
x.dtype                # examine the data type contained within 

dtype('int64')

And we can't necessarily change the data type on the fly by ducktyping (i.e. overwriting the data object with different types of values).

In [47]:
x[1] = .04
x   

array([1, 0, 3, 4])

In [48]:
x[1] = "this"
x

ValueError: invalid literal for int() with base 10: 'this'

To do this, we need to alter the data type of the data contained within the array with `.astype()`

In [49]:
x.astype('f')

array([1., 0., 3., 4.], dtype=float32)

In [50]:
x.astype('U')

array(['1', '0', '3', '4'], dtype='<U21')

List of all data types and their conversions ([table drawn from reading](https://jakevdp.github.io/PythonDataScienceHandbook/02.09-structured-data-numpy.html))

| Character | Description	 | Example  |
|:-------:|:------------:|:--------:|
| `b` | Byte | `np.dtype('b')` |
| `i` | Signed integer | `np.dtype('i4') == np.int32`
| `u` | Unsigned integer | `np.dtype('u1') == np.uint8`
| `f` | Floating point | `np.dtype('f8') == np.int64`
| `c` | Complex floating point | `np.dtype('c16') == np.complex128`
| `S`, `a` | String | `np.dtype('S5')`
| `U` | Unicode string | `np.dtype('U') == np.str_`
| `V` | Raw data (void) | `np.dtype('V') == np.void`

**This limitation extends itself to heterogeneous data types contained in the same array.**

In [51]:
nested_list = [['a','b','c'],[1,2,3],[.3,.55,1.2]]
nested_list

[['a', 'b', 'c'], [1, 2, 3], [0.3, 0.55, 1.2]]

In [52]:
data = np.array(nested_list).T
data

array([['a', '1', '0.3'],
       ['b', '2', '0.55'],
       ['c', '3', '1.2']], dtype='<U4')

All the data in the matrix is treated as a string!

### Structured Arrays
To get around this, we need to again be explicit about the data type of each column. Here we pre-specify a data table and it's inputs.

In [53]:
data = np.zeros((3), dtype={'names':('v1', 'v2', 'v3'),
                            'formats':('U5', 'i', 'f')})
data

array([('', 0, 0.), ('', 0, 0.), ('', 0, 0.)],
      dtype=[('v1', '<U5'), ('v2', '<i4'), ('v3', '<f4')])

We then load the data to the specified columns.

In [54]:
data['v1'] = ['a','b','c']
data['v2'] = [1,2,3]
data['v3'] = [.3,.55,1.2]
data

array([('a', 1, 0.3 ), ('b', 2, 0.55), ('c', 3, 1.2 )],
      dtype=[('v1', '<U5'), ('v2', '<i4'), ('v3', '<f4')])

We can then index, but will do so differently than we observed above. 

In [55]:
data['v1']

array(['a', 'b', 'c'], dtype='<U5')

In [56]:
data[1][['v1','v2']]

('b', 2)

Though possible to deal with heterogeneous data frames using `numpy`, there is a lot of overhead to constructing a data object. As such, we'll use `pandas` `Series` and `DataFrames` to deal with heterogeneous data with less hassle!

# Miscellaneous

### Printing Numpy Arrays

np automatically truncates the data when printing. Handy when you have _alot_ of data

In [ ]:
print(np.arange(10000).reshape(100,100))

In [ ]:
# We can adjust these settings
np.set_printoptions(threshold=None)
print(np.arange(100).reshape(10,10))

### Missing Values

Numpy provides a data class for missing values (i.e. `nan` == "Not a Number", see [here](https://en.wikipedia.org/wiki/NaN))

In [ ]:
Y = np.random.randint(1,10,25).reshape(5,5) + .0
Y

In [ ]:
Y[Y > 5] = np.nan
Y

In [ ]:
type(np.nan)

In [ ]:
# scan for missing values
np.isnan(Y)

In [ ]:
~np.isnan(Y) # are not NAs

When we have missing values, we'll run into issues when computing across the data matrix.

In [ ]:
np.mean(Y)

To get around this, we need to use special version of the methods that compensate for the existence of `nan`.

In [ ]:
np.nanmean(Y)

In [ ]:
np.nanmean(Y,axis=0)

In [ ]:
# Mean impute the missing values
Y[np.where(np.isnan(Y))] = np.nanmean(Y)
Y